In [1]:
import sys
import os
from nltk.tokenize import sent_tokenize,word_tokenize
import nltk
import re

In [2]:
ori_folder = "../data/all/ori"
ann_folder = "../data/all/ann"

In [3]:
type = []
T_cate = []
for filename in os.listdir(ori_folder):
    ori_filepath = os.path.join(ori_folder, filename)
    ann_filepath = os.path.join(ann_folder, filename[:-3]+"ann")
    ori_file = open(ann_filepath,encoding="utf-8")

    for line in open(ann_filepath, 'r', encoding='utf8'):
        type.append(line[0])
        if line[0]=="#":
            print(filename)
        if line.startswith("T"):
            T_cate.append(line.split("\t")[1].split()[0])
    
set(T_cate)

174_Consult-HistoryandPhy.-FlankPain-Consult_8.txt


{'Alcohol',
 'Amount',
 'Drug',
 'EnvironmentalExposure',
 'ExposureHistory',
 'Extent',
 'Family',
 'Frequency',
 'History',
 'InfectiousDiseases',
 'LivingSituation',
 'LivingStatus',
 'Location',
 'MaritalStatus',
 'MedicalCondition',
 'Method',
 'Occupation',
 'Other',
 'PhysicalActivity',
 'QuitHistory',
 'Residence',
 'SexualHistory',
 'Status',
 'Temporal',
 'Tobacco',
 'Type'}

In [5]:
def tokenize_with_positions(text):
    # Sentence tokenize
    sentences = sent_tokenize(text)

    # Word tokenize each sentence and find positions
    tokenized_text = []
    global_offset = 0  # Offset to track position in the entire text
    for sentence in sentences:
        words = word_tokenize(sentence)
        offset = 0  # Offset to track position within the current sentence
        for word in words:
            # Find the position of the word in the sentence
            match = re.search(re.escape(word), sentence[offset:])
            if match:
                start = global_offset + offset + match.start()
                end = global_offset + offset + match.end()
                # Store word with its position (start, end)
                tokenized_text.append((word, start, end))
                offset += match.end()
        
        global_offset += len(sentence) + 1  # +1 for the space or punctuation after the sentence
        tokenized_text.append(())
    return tokenized_text

# Example usage
paragraph = "This is a sentence. Here is another one."
word_positions = tokenize_with_positions(paragraph)
word_positions


[('This', 0, 4),
 ('is', 5, 7),
 ('a', 8, 9),
 ('sentence', 10, 18),
 ('.', 18, 19),
 (),
 ('Here', 20, 24),
 ('is', 25, 27),
 ('another', 28, 35),
 ('one', 36, 39),
 ('.', 39, 40),
 ()]

In [6]:
class BratStandoffReader:
    def __init__(self, file_path):
        self.file_path = file_path
        self.entities = {}
        self.relations = {}
        self.attributes = {}
        self.events = {}
        self._read_file()

    def _read_file(self):
        with open(self.file_path, 'r') as file:
            for line in file:
                self._parse_line(line.strip())

    def _parse_line(self, line):
        if line.startswith('T'):  # Text-bound annotation (Entity)
            self._parse_entity(line)
        elif line.startswith('R'):  # Relation annotation
            self._parse_relation(line)
        elif line.startswith('A'):  # Attribute annotation
            self._parse_attribute(line)
        elif line.startswith('E'):  # Event annotation
            self._parse_event(line)

    def _parse_entity(self, line):
        parts = line.split('\t')
        entity_id = parts[0]
        entity_info = parts[1].split()
        label = entity_info[0]
        start = int(entity_info[1])
        end = int(entity_info[2])
        self.entities[entity_id] = {'label': label, 'start': start, 'end': end, 'text': parts[2]}

    def _parse_relation(self, line):
        parts = line.split()
        relation_id = parts[0]
        label = parts[1]
        arg1 = parts[2].split(':')[1]
        arg2 = parts[3].split(':')[1]
        self.relations[relation_id] = {'label': label, 'arg1': arg1, 'arg2': arg2}

    def _parse_attribute(self, line):
        parts = line.split()
        attribute_id = parts[0]
        label = parts[1]
        entity_id = parts[2]
        value = parts[3] if len(parts) > 3 else None
        self.attributes[attribute_id] = {'label': label, 'entity': entity_id, 'value': value}
        
    def _parse_event(self, line):
        parts = line.split()
        event_id = parts[0]
        event_info = parts[1].split(':')
        trigger = event_info[1]
        args = {part.split(':')[0]: part.split(':')[1] for part in parts[1:] if ':' in part}
        self.events[event_id] = {'trigger': trigger, 'args': args}

    def get_event(self, event_id):
        return self.events.get(event_id)

    def get_entity(self, entity_id):
        return self.entities.get(entity_id)

    def get_relation(self, relation_id):
        return self.relations.get(relation_id)

    def get_attribute(self, attribute_id):
        return self.attributes.get(attribute_id)
    
    def get_pos_with_labelT(self):
        pos_with_label = []
        for i in self.entities:
            pos_with_label.append((self.entities[i]["label"],
                                   self.entities[i]["start"],
                                   self.entities[i]["end"]))
        return pos_with_label
    
    def get_label_T_layer_num(self,layer_num):
        anno_list = self.get_pos_with_labelT()
        mulyer_anno = []
        processed = []

        while len(processed)<len(anno_list):
            layer = []
            occupied=set([])
            for i,a in enumerate(anno_list):
                if i in processed:
                    continue

                if len(set(range(a[1],a[2])) & occupied)==0:

                    occupied=set(range(a[1],a[2])) | occupied
                    layer.append(a)
                    processed.append(i)

            mulyer_anno.append(layer)

        if len(mulyer_anno)>0:
            return mulyer_anno[layer_num]
        else:
            return []
        
    def get_label_T_layer_list2(self):
        anno_list = self.get_pos_with_labelT()
        mulyer_anno = []
        processed = []
        while len(processed)<len(anno_list):
            layer = []
            current = 0
            for i,a in enumerate(anno_list):
                if i in processed:
                    continue
                if a[1]>current:
                    current=a[2]
                    layer.append(a)
                    processed.append(i)

            mulyer_anno.append(layer)

        return mulyer_anno
    
    def get_label_T_layer_list(self):
        anno_list = self.get_pos_with_labelT()
        mulyer_anno = []
        processed = []

        while len(processed)<len(anno_list):
            layer = []
            occupied=set([])
            for i,a in enumerate(anno_list):
                if i in processed:
                    continue

                if len(set(range(a[1],a[2])) & occupied)==0:

                    occupied=set(range(a[1],a[2])) | occupied
                    layer.append(a)
                    processed.append(i)

            mulyer_anno.append(layer)

        return mulyer_anno
    
    

    def mun_of_layerT(self):
        return len(self. get_label_T_layer_list())
    
    
    
    

In [62]:
def single_layer_data(ori_folder,ann_folder,output_path):
    all_token_merged = open(output_path,"w",encoding="utf-8")
    for filename in os.listdir(ori_folder):
        ori_filepath = ori_folder+"/"+filename
        ann_filepath = ann_folder+"/"+filename[:-3]+"ann"
        with open(ori_filepath, 'r', encoding='utf8') as f:
            ori_sents = f.read().strip()
            tokens = tokenize_with_positions(ori_sents)

        annos = BratStandoffReader(ann_filepath)
        anno_list = annos.get_label_T_layer_num(0)

        for token in tokens:
            if len(token)==0:
                all_token_merged.write("\n")
            else:
                label = ""
                word = token[0]
                w_start = token[1]
                w_end = token[2]

                for anno in anno_list:
                    ann = anno[0]
                    a_start = anno[1]
                    a_end = anno[2]

                    if w_start == a_start:
                        label += "B-"+ann
                    elif w_start > a_start and w_end<a_end:
                        label = "I-"+ann
                    elif w_end==a_end:
                        label = "I-"+ann
                if len(label)<1:
                    label = "O"
                if label.count("-")>1:
                    label = "O"

                all_token_merged.write(word +" "+ label+"\n")    
    all_token_merged.close()
    return anno_list


ori_folder = "../data/train/ori"
ann_folder = "../data/train/ann"
output_path = "../data/train/training_data/train_single_layer.txt"
single_layer_data(ori_folder,ann_folder,output_path)

[]

In [64]:
ori_folder = "../data/test/ori"
ann_folder = "../data/test/ann"
output_path = "../data/test/test_data/test_single_layer.txt"
single_layer_data(ori_folder,ann_folder,output_path)

[]

In [65]:
ori_folder = "../data/valid/ori"
ann_folder = "../data/valid/ann"
output_path = "../data/valid/valid_data/valid_single_layer.txt"
single_layer_data(ori_folder,ann_folder,output_path)

[('MaritalStatus', 18, 25),
 ('Status', 27, 36),
 ('Status', 42, 50),
 ('Alcohol', 51, 66),
 ('Status', 68, 81),
 ('Type', 82, 94),
 ('Drug', 95, 104)]

In [32]:
def mulit_layer_data(ori_folder,ann_folder,output_path):
    all_token_merged = open(output_path,"w",encoding="utf-8")
    for filename in os.listdir(ori_folder):
        ori_filepath = ori_folder+"/"+filename
        ann_filepath = ann_folder+"/"+filename[:-3]+"ann"
        with open(ori_filepath, 'r', encoding='utf8') as f:
            ori_sents = f.read().strip()
            tokens = tokenize_with_positions(ori_sents)
        

        annos = BratStandoffReader(ann_filepath)
        anno_list = annos.get_label_T_layer_num(0)
        multi_anno = annos.get_label_T_layer_list()
        #print(ori_filepath)
        for token in tokens:
            if len(token)==0:
                all_token_merged.write("\n")
                #print()

            else:
                all_label = ""
                for anno_list in multi_anno:
                    word = token[0]
                    w_start = token[1]
                    w_end = token[2]
                    label = ""

                    for anno in anno_list:
                        ann = anno[0]
                        a_start = anno[1]
                        a_end = anno[2]

                        if w_start == a_start:
                            label += "B-"+ann+" "
                        elif w_start > a_start and w_end<a_end:
                            label = "I-"+ann+" "
                        elif w_end==a_end:
                            label = "I-"+ann+" "
                    if len(label)<1:
                        label = "O "
                    all_label+=label

                
                
                if len(all_label)>0:
                    #print(word +" "+ all_label)
                    all_token_merged.write(word +" "+ all_label+"\n")
    all_token_merged.close()




ori_folder = "../data/train/ori"
ann_folder = "../data/train/ann"
output_path = "../data/train/training_data/train_multi_layer.txt"
mulit_layer_data(ori_folder,ann_folder,output_path)

In [33]:
ori_folder = "../data/test/ori"
ann_folder = "../data/test/ann"
output_path = "../data/test/test_data/test_multi_layer.txt"
mulit_layer_data(ori_folder,ann_folder,output_path)

In [34]:
ori_folder = "../data/valid/ori"
ann_folder = "../data/valid/ann"
output_path = "../data/valid/valid_data/valid_multi_layer.txt"
mulit_layer_data(ori_folder,ann_folder,output_path)